In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import argparse
import sys
import os

import csv
import zipfile
import xml.etree.ElementTree as ET

from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
def convert_to_csv(path, name, directory):
    """Read XML dataset"""
    with zipfile.ZipFile(path, mode='r') as z:
        with z.open([filename for filename in z.namelist() if filename[-4:] == '.xml'][0]) as f:
            tree = ET.parse(f)
            root = tree.getroot()
            
            filename = os.path.join(directory, name + '.csv')

            with open(filename, 'wt', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile, delimiter=',')

                for Question in root:
                    QID = int(Question.get('QID'))
                    Qtext = Question.find('Qtext').text

                    for QApair in Question.iter('QApair'): 
                        QAID = int(QApair.get('QAID'))
                        QArel = QApair.get('QArel')
                        QAconf = float(QApair.get('QAconf'))
                        QAquestion = QApair.find('QAquestion').text
                        QAanswer = QApair.find('QAanswer').text
                        
                        writer.writerow([QID, Qtext, QAID, QArel, QAconf, QAquestion, QAanswer])

                    

In [4]:
convert_to_csv('D:\PhD\SemEval\TRAIN\SemEval2016-Task3-CQA-MD-train.xml.zip', 'train', 'temp')

In [5]:
MAX_DOCUMENT_LENGTH = 10
EMBEDDING_SIZE = 50
n_words = 0
MAX_LABEL = 15
WORDS_FEATURE = 'words'  # Name of the input words feature.

In [6]:
def estimator_spec_for_softmax_classification(
    logits, labels, mode):
    """Returns EstimatorSpec instance for softmax classification."""
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'class': predicted_classes,
                'prob': tf.nn.softmax(logits)
            })

    onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      'accuracy': tf.metrics.accuracy(
          labels=labels, predictions=predicted_classes)
    }
    
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [7]:
def bag_of_words_model(features, labels, mode):
    """A bag-of-words model. Note it disregards the word order in the text."""
    bow_column = tf.feature_column.categorical_column_with_identity(
      WORDS_FEATURE, num_buckets=n_words)
    bow_embedding_column = tf.feature_column.embedding_column(
      bow_column, dimension=EMBEDDING_SIZE)
    bow = tf.feature_column.input_layer(
      features,
      feature_columns=[bow_embedding_column])
    logits = tf.layers.dense(bow, MAX_LABEL, activation=None)

    return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)

In [8]:
def rnn_model(features, labels, mode):
    """RNN model to predict from sequence of words to a class."""
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

    # Split into list of embedding per word, while removing doc length dim.
    # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
    word_list = tf.unstack(word_vectors, axis=1)

    # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    # Create an unrolled Recurrent Neural Networks to length of
    # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    # Given encoding of RNN, take encoding of last step (e.g hidden size of the
    # neural network of last step) and pass it as features for softmax
    # classification over output classes.
    logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
    return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)

In [12]:
training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename='temp/train.csv',
    features_dtype=np.str,
    target_dtype=np.int32,
    target_column=4)

x_train = pd.Series(training_set.data[:,1])
y_train = pd.Series(training_set.target)

ValueError: invalid literal for int() with base 10: '1.0'

In [10]:
# Process vocabulary
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
  MAX_DOCUMENT_LENGTH)

x_transform_train = vocab_processor.fit_transform(x_train)
x_train = np.array(list(x_transform_train))

n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 7456


In [11]:
classifier = tf.estimator.Estimator(model_fn=bag_of_words_model)

# Train.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={WORDS_FEATURE: x_train},
  y=y_train,
  batch_size=len(x_train),
  num_epochs=None,
  shuffle=True)
classifier.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_session_config': None, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Yassine\\AppData\\Local\\Temp\\tmp4n6tjs5o', '_save_checkpoints_secs': 600}


TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: uint8, int32, int64

In [13]:
dbpedia = tf.contrib.learn.datasets.load_dataset('dbpedia')

In [14]:
dbpedia.train.target

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6